In [1]:
import datetime
import scipy.stats as stats
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
matplotlib.use('Agg')
%matplotlib inline
from dateutil.relativedelta import relativedelta

from finrl import config
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline, get_baseline_tdx,convert_daily_return_to_pyfolio_ts

import torch
# import plotly.express as px

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import EfficientFrontier
from pypfopt import expected_returns
from pypfopt import objective_functions
from pyfolio import timeseries

import plotly.graph_objects as go
from plotly.subplots import make_subplots

########
import backtrader as bt
import backtrader.indicators as btind
import backtrader.analyzers as btanalyzers

import pyfolio as pf
from matplotlib import rcParams

d:\code\python\pyfolio\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [2]:
from lutils.stock import LTdxHq

In [3]:
train_start = '2019-01-01'
train_end = '2022-11-13'
trade_start = train_end
trade_end = '2022-11-22'

In [4]:
stock_count = 4

In [5]:
# ddf = pd.read_pickle('d:/ddf.pkl').dropna()

ddf = pd.read_pickle('d:/ddf_1127.pkl').dropna()
# ddf = pd.read_pickle('d:/ddf_0606.pkl').dropna()

In [6]:
ddf = ddf.sort_values(['date','tic'], ignore_index=True)
ddf.index = ddf.date.factorize()[0]

In [7]:
days = ddf.date.unique()
days.shape

(1190,)

In [18]:
days

array(['2018-01-02T00:00:00.000000000', '2018-01-03T00:00:00.000000000',
       '2018-01-04T00:00:00.000000000', ...,
       '2022-11-23T00:00:00.000000000', '2022-11-24T00:00:00.000000000',
       '2022-11-25T00:00:00.000000000'], dtype='datetime64[ns]')

In [24]:
days[np.where(days==np.datetime64('2022-11-25'))[0][0]]

numpy.datetime64('2022-11-25T00:00:00.000000000')

In [8]:
dl = ddf[(ddf['date'] > days[-63]) & (ddf['date'] < trade_start)]
dd = dl.pivot_table(index = 'date',columns = 'tic', values = 'close').pct_change().dropna()
corr = dd.cov().corr()
# corr.style.background_gradient(cmap='coolwarm')

# c1 = corr.abs().unstack().sort_values(ascending = True)
# c1 = corr.unstack().sort_values(ascending = True)

In [12]:
# corr.style.background_gradient(cmap='coolwarm')

In [13]:
# corr.unstack().sort_values().index.get_level_values(0)[:10]

In [14]:
corr.sum().sort_values() # ascending=False

tic
601288    -786.948491
601398    -733.569122
601939    -688.925879
002541    -657.358337
601988    -616.878697
             ...     
300067    1534.498786
300405    1536.302348
002367    1538.242585
603686    1541.522359
300243    1554.217592
Length: 2790, dtype: float64

In [15]:
# stock_codes = list(np.random.choice(ddf.tic.unique(), 6))

In [16]:
# corr.unstack().sort_values().index.get_level_values(1)

In [17]:
# corr.style.background_gradient(cmap='coolwarm')

In [54]:
_stock_codes = list(corr.unstack().sort_values().index.get_level_values(0)[:stock_count * 2])
stock_codes = []
while len(stock_codes) < stock_count:
    sc = _stock_codes.pop(0)
    if sc not in stock_codes:
        stock_codes.append(sc)

# ['601398', '000738']
# ['601988', '603538'] # 2022-02-08
# ['600377', '000887'] # 2022-02-14
# ['301040', '000573'] # 2022-02-15
# ['301040', '000573', '300384'] # 2022-02-20

In [19]:
# stock_codes = list(corr.sum().sort_values().index.values[:2]) # ascending=False
# ['000921', '600365', '000609', '601868', '002372', '001965']
# ['601868', '000501', '603789', '002311', '002873', '000921']
# ['000921', '603789', '002372', '601868', '300675', '002032']
# ['600775', '603789', '300622', '603228', '600172', '001965']

In [20]:
# stock_codes = ['601186', '002645', '600546', '688222', '600107', '603028']
# stock_codes = ['601988', '603538']

# ['603003', '603326', '002533', '600459']

In [21]:
stock_codes

['300516', '601288', '002698', '300183']

In [23]:
ltdxhq = LTdxHq()

indexs = None
dfs = []
for code in stock_codes:
    df = ltdxhq.get_k_data_daily(code, start='2018-01-01') # 2014-01-01
    
    if indexs is None:
        indexs = df.index
    else:
        indexs = indexs.union(df.index)
    
#     df = df.assign(date = df.index)
#     df = df.assign(day = df.index.weekday)
#     df.date = df.date.dt.strftime('%Y-%m-%d')
    df = df.assign(tic = code)
#     df.index = range(df.shape[0])
    
    dfs.append(df)
    print('----------- over %s min: %s max: %s -----------' % (code, df.index.min(), df.index.max()))

for i, df in enumerate(dfs):
    df = df.reindex(indexs)
    df = df.assign(date = df.index)
    df = df.assign(day = df.index.weekday)
    df.index = range(df.shape[0])
    
    dfs[i] = df.ffill()
    
# df = pd.concat(dfs)
# df.index = range(df.shape[0])

ltdxhq.close()

----------- over 300516 min: 2018-01-02 00:00:00 max: 2022-11-25 00:00:00 -----------
----------- over 601288 min: 2018-01-02 00:00:00 max: 2022-11-25 00:00:00 -----------
----------- over 002698 min: 2018-01-02 00:00:00 max: 2022-11-25 00:00:00 -----------
----------- over 300183 min: 2018-01-02 00:00:00 max: 2022-11-25 00:00:00 -----------


In [24]:
df = pd.concat(dfs)

In [29]:
df

,open,close,high,low,volume,amount,tic,date,day
0,29.82,29.92,29.94,29.55,311300.0,14106950.0,300516,2018-01-02,1
1,29.88,30.09,30.20,29.71,308938.0,14126594.0,300516,2018-01-03,2
2,30.07,29.54,30.07,29.50,388750.0,17598828.0,300516,2018-01-04,3
3,29.63,29.70,29.82,29.29,330300.0,14894724.0,300516,2018-01-05,4
4,29.77,29.61,29.77,29.34,330300.0,14821094.0,300516,2018-01-08,0
...,...,...,...,...,...,...,...,...,...
1185,13.92,14.05,14.13,13.85,2357281.0,33029980.0,300183,2022-11-21,0
1186,14.04,13.82,14.08,13.75,2626381.0,36554276.0,300183,2022-11-22,1
1187,13.78,13.57,13.83,13.30,3726462.0,50390360.0,300183,2022-11-23,2
1188,13.62,13.49,13.77,13.41,1945967.0,26368242.0,300183,2022-11-24,3


In [48]:
class StrategyText(bt.Strategy):
    
#     params = (('period', 5),)
    
    def __init__(self):
#         self.sma_turnover_rate = btind.SMA(self.data0.turnover_rate, period=self.p.period, plot=False)
#         self.sma_turnover_rate15 = btind.SMA(self.data0.turnover_rate, period=7, plot=False)
#         self.crossover = btind.CrossOver(self.sma_turnover_rate, self.sma_turnover_rate15)
        pass

    def next(self):
#         if not self.position:
#             if self.crossover > 0:
#                 self.buy()
#         elif not self.crossover < 0:
#             self.close()
        
#         if not self.position:
#             if self.data0.close > self.sma_turnover_rate:
#                 self.buy()
#         elif self.data0.close < self.sma_turnover_rate:
#             self.close()

        if not self.position:
            if self.data0.turnover_rate / self.sma_turnover_rate[-1] > 2.5 and self.data0.change_rate < -4:
                self.buy()
        elif self.data0.turnover_rate / self.sma_turnover_rate[-1] < 1.5:
            self.close()
            
        

IndentationError: expected an indented block (<ipython-input-48-baac3efd033e>, line 11)

In [38]:
# import matplotlib.pyplot as plt
# import squarify 
# %matplotlib inline
# sizes=[40, 30, 5, 25]
# label=["A", "B", "C", "D"]
# squarify.plot(sizes=sizes, label=label, alpha=0.6)
# plt.show()

In [ ]:
buy_money = 5000 * len(stocks_test)

In [61]:
# stocks_test = ['002533', '600459', '603003', '603326']
# pers = [0.25, 0.25, 0.25, 0.25]

# stocks_test = ['300652', '600900', '601398']
# pers = [.3333, .3333, .3333]

# stocks_test = ['300516', '601288', '002698', '300183'] # ['300395', '600000', '601988', '603040']
stocks_test = ['300516', '601288', '002698', '300183', '601988', ] # '300570', ] #'000534', '601939']
# pers = [0.25, 0.25, 0.25, 0.25]
pers = [1 / len(stocks_test)] * len(stocks_test)


ltdxhq = LTdxHq()

indexs = None
dfs = []
left_money = 0
after_money = 0
keeps = []
for code, per in zip(stocks_test, pers):
    df = ltdxhq.get_k_data_daily(code, start=trade_start) #, end=trade_end) # 2014-01-01
    keep = (buy_money * per / (df.iloc[0].close * 100))
    keeps.append(keep)
    
    left_money += buy_money * per - keep * df.iloc[0].close * 100
    
    after_money += df.iloc[-1].close * 100 * keep
    
after_money += left_money
    

In [62]:
buy_money = after_money

25406.759255208737